# Benchmarking Notebook

This notebook contains fine tuning a model by just merging all text together

In [2]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('chianglab-dataderivatives')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)

length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [4]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
# df = df.drop("eddischarge_category",axis=1)
df['all'] = df['arrival'] + df['pyxis'] + df['triage'] + df['medrecon'] + df['vitals'] + df['codes']
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,arrival,triage,medrecon,vitals,pyxis,codes,all,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...","At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10014729, a 21 year old white - other ...",1
34176810,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient did not receive any medications.,The patient received the following diagnostic ...,"Patient 10018328, a 83 year old white female, ...",1
32103106,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10018328, a 83 year old white female, ...",0
38797992,"Patient 10020640, a 91 year old white female, ...","At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10020640, a 91 year old white female, ...",1
33473053,"Patient 10015272, a 78 year old white female, ...","At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10015272, a 78 year old white female, ...",1
...,...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10038999, a 45 year old white male, ar...",1
31628990,"Patient 10009049, a 56 year old white male, ar...","At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10009049, a 56 year old white male, ar...",1
32405286,"Patient 10004457, a 65 year old white male, ar...","At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10004457, a 65 year old white male, ar...",1
34391979,"Patient 10004720, a 61 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10004720, a 61 year old white male, ar...",1


In [5]:
df = df.rename(columns={"eddischarge": "label"})
df['ID'] = df.arrival.astype(str).str.split().str[1].replace(",", " ", regex=True).to_list()
df

,arrival,triage,medrecon,vitals,pyxis,codes,all,label,ID
37887480,"Patient 10014729, a 21 year old white - other ...","At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10014729, a 21 year old white - other ...",1,10014729
34176810,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient did not receive any medications.,The patient received the following diagnostic ...,"Patient 10018328, a 83 year old white female, ...",1,10018328
32103106,"Patient 10018328, a 83 year old white female, ...","At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10018328, a 83 year old white female, ...",0,10018328
38797992,"Patient 10020640, a 91 year old white female, ...","At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10020640, a 91 year old white female, ...",1,10020640
33473053,"Patient 10015272, a 78 year old white female, ...","At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10015272, a 78 year old white female, ...",1,10015272
...,...,...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10038999, a 45 year old white male, ar...",1,10038999
31628990,"Patient 10009049, a 56 year old white male, ar...","At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10009049, a 56 year old white male, ar...",1,10009049
32405286,"Patient 10004457, a 65 year old white male, ar...","At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10004457, a 65 year old white male, ar...",1,10004457
34391979,"Patient 10004720, a 61 year old white male, ar...","At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,"Patient 10004720, a 61 year old white male, ar...",1,10004720


In [6]:
# split dataframe here
def train_validate_test_split(df, train_percent=.7, validate_percent=.15, seed=None):
    np.random.seed(seed)
    df = df.reset_index()
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    train = train.set_index('index')
    validate = validate.set_index('index')
    test = test.set_index('index')
    return train, validate, test

In [7]:
# t, val, t2 = train_validate_test_split(df, seed=7)
# remain = pd.concat([val, t2])
# print("Size of validation + test after concat: ", len(remain)) # sanity check

#resplit the our testing dataframe into an additional train and test split for fine tuning 
train, validate, test =  train_validate_test_split(df, seed=7)
print("98% Train:",len(train), "\n2% Test:",len(validate+test))

98% Train: 147 
2% Test: 63


In [8]:
patient_IDS = test.ID.to_list()

In [9]:
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# tokenize
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
def tokenize(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["all"], truncation=True, padding="max_length",
                    max_length=512, return_special_tokens_mask=True)

train = Dataset.from_pandas(train)
validate = Dataset.from_pandas(validate)
test = Dataset.from_pandas(test)

train_set = train.map(tokenize, batched=True)
validate_set = validate.map(tokenize, batched=True)
test_set = test.map(tokenize, batched=True)

train_set.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
validate_set.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
test_set.set_format('torch', columns=["input_ids", "attention_mask", "label"] )

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

dataset_cc = DatasetDict({
    'train': train_set,
    'test': validate_set,
    'valid': test_set})

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset_cc['train'], shuffle = True, batch_size = 1, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    dataset_cc['valid'], shuffle = True, collate_fn = data_collator
)

test_dataloader = DataLoader(
    dataset_cc['test'], collate_fn = data_collator
)

In [13]:
class EDdisposition(nn.Module):
    """
    A task-specific custom transformer model for predicting ED Disposition. 
    This model loads a pre-trained transformer model and adds a new dropout 
    and linear layer at the end for fine-tuning and prediction on specific tasks.
    """
    def __init__(self, checkpoint, num_labels ):
        """
        Args:
            checkpoint (str): The name of the pre-trained model or path to the model weights.
            num_labels (int): The number of output labels in the final classification layer.
        """
        super(EDdisposition, self).__init__()
        self.num_labels = num_labels # number of labels for classifier
        
        # checkpoint is the model name 
        self.model = model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint, 
                                                                                                       output_attention = True, 
                                                                                                       output_hidden_state = True ) )
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels) #FC Layer
        
    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        """
        Forward pass for the model.
        
        Args:
            input_ids (torch.Tensor, optional): Tensor of input IDs. Defaults to None.
            attention_mask (torch.Tensor, optional): Tensor for attention masks. Defaults to None.
            labels (torch.Tensor, optional): Tensor for labels. Defaults to None.
            
        Returns:
            TokenClassifierOutput: A named tuple with the following fields:
            - loss (torch.FloatTensor of shape (1,), optional, returned when label_ids is provided) – Classification loss.
            - logits (torch.FloatTensor of shape (batch_size, num_labels)) – Classification scores before SoftMax.
            - hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) – Tuple of torch.FloatTensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
            - attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) – Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).
        """
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask  )
        
        last_hidden_state = outputs[0]
        
        sequence_outputs = self.dropout(last_hidden_state)
        
        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))
        
        loss = None
        if labels is not None:
            loss_func = nn.CrossEntropyLoss() # Change this if it becomes more than binary classification
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
            
            # TokenClassifierOutput
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
        

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_task_specific = EDdisposition(checkpoint=model, num_labels=2 ).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model_task_specific.parameters(), lr = 5e-5 )

num_epoch = 5

num_training_steps = num_epoch * len(dataset_cc['test']["input_ids"])

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
from datasets import load_metric
metric = load_metric("f1", "precision")

<ipython-input-17-f7082ec09936>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1", "precision")


In [20]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader) ))


for epoch in range(num_epoch):
    model_task_specific.train()
    for batch in train_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model_task_specific(**batch)
        loss = outputs.loss
        loss.backward() # computes gradients
        
        optimizer.step() # updates the weights and biases based on these gradients
        lr_scheduler.step() # updates the weights and biases based on these gradients
        optimizer.zero_grad() # used to clear the gradients of all parameters in a model
        # progress_bar_train.update(1)
    print("epoch training", str(epoch), "done")
    print("loss:", str(loss))
    
    # run on validation set
    model_task_specific.eval()
    for batch in eval_dataloader:
        #batch = { k: v.to(device) for k, v in batch.items() }
        with torch.no_grad(): # a context manager that disables gradient calculation during model inference
            outputs = model_task_specific(**batch)
            
        logits = outputs.logits # calculates the probabilities between the labels
        predictions = torch.argmax(logits, dim = -1 ) # takes the label closest to 1
        metric.add_batch(predictions = predictions, references = batch['labels'] ) 
        # progress_bar_eval.update(1)
        
    print(metric.compute()) 



  0%|          | 0/160 [00:35<?, ?it/s]


epoch training 0 done
loss: tensor(0.5288, grad_fn=<NllLossBackward0>)
{'f1': 0.7692307692307693}
epoch training 1 done
loss: tensor(0.6571, grad_fn=<NllLossBackward0>)
{'f1': 0.7692307692307693}
epoch training 2 done
loss: tensor(0.7424, grad_fn=<NllLossBackward0>)
{'f1': 0.7692307692307693}
epoch training 3 done
loss: tensor(0.5073, grad_fn=<NllLossBackward0>)
{'f1': 0.7692307692307693}
epoch training 4 done
loss: tensor(0.6754, grad_fn=<NllLossBackward0>)
{'f1': 0.7692307692307693}


In [21]:
with open('./models/data/results_merge_dataset.txt', 'w') as file:
    
    test_dataloader = DataLoader(
        dataset_cc['test'], batch_size = 1, collate_fn = data_collator
    )    
    file.write("patient      predicted     true\n")
    for i, batch in enumerate(test_dataloader):
        file.write(patient_IDS[i]+"     ")
        batch = { k: v.to(device) for k, v in batch.items() }
        with torch.no_grad(): # a context manager that disables gradient calculation during model inference
            outputs = model_task_specific(**batch)

        logits = outputs.logits # calculates the probabilities between the labels
        predictions = torch.argmax(logits, dim = -1 ) # takes the label closest to 1
        metric.add_batch(predictions = predictions, references = batch['labels'] )
        if predictions == 0:
            file.write("home         ")
        else: 
            file.write("admitted     ")
        if batch['labels'] == 0:
            file.write("home\n")
        else: 
            file.write("admitted\n")

    print(metric.compute()) 

{'f1': 0.8301886792452831}
